In [1]:
# Import dependencies
import pandas as pd
import sqlalchemy
from sqlalchemy import create_engine, func
import numpy as np
import sqlite3


In [2]:
# create engine to historical prices sqlite db
engine = create_engine("sqlite:///test.sqlite")

In [3]:
eur_usd_df = pd.read_sql('select * from eur_usd',engine)
cad_usd_df = pd.read_sql('select * from cad_usd', engine)
jpy_usd_df = pd.read_sql('select * from jpy_usd', engine)

In [4]:
eur_usd_df

,Date,Open,High,Low,Close,MA5,MA8,MA13,Buy,Profit
0,01/01/24,1.1058,1.1080,1.1037,1.1047,NaN,NaN,NaN,Closed,0.0
1,01/02/24,1.1046,1.1056,1.0938,1.0940,NaN,NaN,NaN,Closed,0.0
2,01/03/24,1.0937,1.0967,1.0895,1.0922,NaN,NaN,NaN,Closed,0.0
3,01/04/24,1.0920,1.0975,1.0915,1.0946,NaN,NaN,NaN,Closed,0.0
4,01/05/24,1.0945,1.0998,1.0877,1.0939,NaN,NaN,NaN,Closed,0.0
...,...,...,...,...,...,...,...,...,...,...
258,12/26/24,1.0407,1.0432,1.0381,1.0425,1.04020,1.042150,1.045562,Closed,0.0
259,12/27/24,1.0419,1.0446,1.0406,1.0428,1.04146,1.041037,1.044562,Closed,0.0
260,12/30/24,1.0427,1.0459,1.0373,1.0408,1.04142,1.040225,1.043777,Closed,0.0
261,12/31/24,1.0409,1.0426,1.0345,1.0355,1.04144,1.040887,1.043092,Closed,0.0


In [5]:
# eur_usd_df[' Close']
def MA5(table):
    counter = 0
    avg5_list = []
    for x in table[' Close']:
        if(counter>4):
            avg5 = np.average([table[' Close'][counter - 5],
                               table[' Close'][counter - 4],
                               table[' Close'][counter - 3],
                               table[' Close'][counter - 2],
                               table[' Close'][counter - 1]])
            avg5_list.append(avg5)
        else:
            avg5_list.append(None)
        counter += 1

    return avg5_list


In [6]:
eur_usd_df[' MA5'] = MA5(eur_usd_df)
cad_usd_df[' MA5'] = MA5(cad_usd_df)
jpy_usd_df[' MA5'] = MA5(jpy_usd_df)
# print(jpy_usd_df[0:10])

In [7]:
def MA8(table):
    counter = 0
    avg8_list = []
    for x in table[' Close']:
        if(counter>7):
            avg8 = np.average([table[' Close'][counter - 8],
                               table[' Close'][counter - 7],
                               table[' Close'][counter - 6],
                               table[' Close'][counter - 5],
                               table[' Close'][counter - 4],
                               table[' Close'][counter - 3],
                               table[' Close'][counter - 2],
                               table[' Close'][counter - 1]])
            avg8_list.append(avg8)
        else:
            avg8_list.append(None)
        counter += 1

    return avg8_list

In [8]:
eur_usd_df[' MA8'] = MA8(eur_usd_df)
cad_usd_df[' MA8'] = MA8(cad_usd_df)
jpy_usd_df[' MA8'] = MA8(jpy_usd_df)
# jpy_usd_df[0:12]

In [9]:
def MA13(table):
    counter = 0
    avg13_list = []
    for x in table[' Close']:
        if(counter>12):
            avg13 = np.average([table[' Close'][counter - 13],
                                table[' Close'][counter - 12],
                                table[' Close'][counter - 11],
                                table[' Close'][counter - 10],
                                table[' Close'][counter - 9],
                                table[' Close'][counter - 8],
                                table[' Close'][counter - 7],
                                table[' Close'][counter - 6],
                                table[' Close'][counter - 5],
                                table[' Close'][counter - 4],
                                table[' Close'][counter - 3],
                                table[' Close'][counter - 2],
                                table[' Close'][counter - 1]])
            avg13_list.append(avg13)
        else:
            avg13_list.append(None)
        counter += 1

    return avg13_list

In [10]:
eur_usd_df[' MA13'] = MA13(eur_usd_df)
cad_usd_df[' MA13'] = MA13(cad_usd_df)
jpy_usd_df[' MA13'] = MA13(jpy_usd_df)
# jpy_usd_df[0:15]

In [11]:
def setup_buy_col(table):
        buy = []
        for row in table.iterrows():
                buy.append('Closed')
        return buy

def setup_sell_col(table):
        sell = []
        for row in table.iterrows():
                sell.append('Closed')
        return sell

def setup_prof_col(table):
        profit = []
        for row in table.iterrows():
                profit.append(0)
        return profit


In [12]:
eur_usd_df['Buy'] = setup_buy_col(eur_usd_df)
eur_usd_df['Sell'] = setup_sell_col(eur_usd_df)
eur_usd_df['Profit'] = setup_prof_col(eur_usd_df)

cad_usd_df['Buy'] = setup_buy_col(cad_usd_df)
cad_usd_df['Sell'] = setup_sell_col(cad_usd_df)
cad_usd_df['Profit'] = setup_prof_col(cad_usd_df)

jpy_usd_df['Buy'] = setup_buy_col(jpy_usd_df)
jpy_usd_df['Sell'] = setup_sell_col(jpy_usd_df)
jpy_usd_df['Profit'] = setup_prof_col(jpy_usd_df)

In [13]:
def algo_buy(table):
    for index, row in table.iterrows():
        if (index>12):
            if (table['Buy'][index-1] == 'Closed'): # in closed position, want to check if should open position
                if ((table[' MA5'][index] > table[' MA8'][index]) & (table[' MA8'][index] > table[' MA13'][index])):
                    table['Buy'][index] = 'Open'

            if (table['Buy'][index-1] == 'Open'): # in open position, want to check if should close position
                if (table[' Close'][index] < table[' MA13'][index]):
                    table['Buy'][index] = 'Closed'
                else:
                    table['Buy'][index] = 'Open'
    return table['Buy']

In [14]:
eur_usd_df['Buy'] = algo_buy(eur_usd_df)
cad_usd_df['Buy'] = algo_buy(cad_usd_df)
jpy_usd_df['Buy'] = algo_buy(jpy_usd_df)

/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_69551/77332307.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Buy'][index] = 'Open'
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_69551/77332307.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Buy'][index] = 'Open'
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_69551/77332307.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t

In [15]:
def algo_sell(table):
    for index, row in table.iterrows():
        if (index>12):
            if (table['Sell'][index-1] == 'Closed'): # in closed position, want to check if should open position
                if ((table[' MA5'][index] < table[' MA8'][index]) & (table[' MA8'][index] < table[' MA13'][index])):
                    table['Sell'][index] = 'Open'

            if (table['Sell'][index-1] == 'Open'): # in open position, want to check if should close position
                if (table[' Close'][index] > table[' MA5'][index]):
                    table['Sell'][index] = 'Closed'
                else:
                    table['Sell'][index] = 'Open'
    return table['Sell']

In [16]:
eur_usd_df['Sell'] = algo_sell(eur_usd_df)
cad_usd_df['Sell'] = algo_sell(cad_usd_df)
jpy_usd_df['Sell'] = algo_sell(jpy_usd_df)

/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_69551/2913671093.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Sell'][index] = 'Open'
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_69551/2913671093.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Sell'][index] = 'Open'
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_69551/2913671093.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-

In [17]:
eur_usd_df[58:70]

,Date,Open,High,Low,Close,MA5,MA8,MA13,Buy,Profit,Sell
58,03/21/24,1.0923,1.0944,1.0856,1.0861,1.08868,1.090500,1.090323,Closed,0,Closed
59,03/22/24,1.0859,1.0870,1.0801,1.0810,1.08826,1.089687,1.090362,Closed,0,Open
60,03/25/24,1.0864,1.0864,1.0801,1.0838,1.08666,1.088188,1.089992,Closed,0,Open
61,03/26/24,1.0839,1.0865,1.0824,1.0831,1.08598,1.086788,1.089531,Closed,0,Open
62,03/27/24,1.0833,1.0840,1.0811,1.0828,1.08524,1.086150,1.088615,Closed,0,Open
63,03/28/24,1.0826,1.0830,1.0775,1.0788,1.08336,1.085375,1.087754,Closed,0,Open
64,03/29/24,1.0790,1.0804,1.0768,1.0797,1.08190,1.084325,1.086692,Closed,0,Open
65,04/01/24,1.0801,1.0801,1.0731,1.0743,1.08164,1.083438,1.085669,Closed,0,Open
66,04/02/24,1.0744,1.0780,1.0724,1.0770,1.07974,1.081200,1.084077,Closed,0,Open
67,04/03/24,1.0771,1.0838,1.0763,1.0836,1.07852,1.080062,1.083215,Closed,0,Closed


In [18]:
for index, row in eur_usd_df.iterrows():
    print(row[' Close'])

1.1047
1.094
1.0922
1.0946
1.0939
1.0952
1.0933
1.0975
1.0972
1.0951
1.0953
1.0878
1.0883
1.0877
1.0897
1.0884
1.0855
1.0886
1.0849
1.0855
1.0834
1.0845
1.0819
1.0872
1.0788
1.0743
1.0755
1.0774
1.0778
1.0783
1.0773
1.0709
1.0729
1.0773
1.0778
1.078
1.0809
1.0818
1.0824
1.0822
1.0854
1.0845
1.0839
1.0807
1.084
1.0856
1.0858
1.0898
1.095
1.094
1.0926
1.093
1.095
1.0882
1.089
1.0872
1.0868
1.0922
1.0861
1.081
1.0838
1.0831
1.0828
1.0788
1.0797
1.0743
1.077
1.0836
1.0837
1.0839
1.0861
1.0857
1.0744
1.0728
1.0641
1.0624
1.0618
1.0674
1.0643
1.0657
1.0655
1.0702
1.07
1.0731
1.0693
1.0721
1.0667
1.0715
1.0727
1.0763
1.0769
1.0756
1.075
1.0782
1.0773
1.079
1.0819
1.0884
1.0869
1.0871
1.0857
1.0855
1.0824
1.0816
1.0844
1.0858
1.0857
1.0802
1.0834
1.0848
1.0904
1.0881
1.0871
1.0889
1.0817
1.0767
1.074
1.0809
1.0736
1.0704
1.0735
1.0741
1.0745
1.0701
1.0695
1.0736
1.0716
1.068
1.0704
1.0716
1.074
1.0747
1.079
1.0813
1.0839
1.0825
1.0813
1.083
1.0869
1.0908
1.0896
1.0899
1.0939
1.0897
1.0884
1.08

In [19]:
def profit(table):
    open_price = 0
    closed_price = 0
    # buy profit
    for index, row in table.iterrows():
        if (index>12):
            if ((table['Buy'][index] == 'Closed') & (table['Buy'][index - 1] == 'Closed')): # we're in a closed postion
                table['Profit'][index] = 0
            elif ((table['Buy'][index] == 'Closed') & (table['Buy'][index - 1] == 'Open')): # we just closed a postition
                closed_price = table[' Close'][index]
                table['Profit'][index] = closed_price - open_price
            elif ((table['Buy'][index] == 'Open') & (table['Buy'][index - 1] == 'Open')): # we're in an open postion
                table['Profit'][index] = 0
            elif ((table['Buy'][index] == 'Open') & (table['Buy'][index - 1] == 'Closed')): # we just opend a postion
                open_price = table[' Close'][index]

    open_price = 0
    closed_price = 0
    # sell profit
    for index, row in table.iterrows():
        if (index>12):
            if ((table['Sell'][index] == 'Closed') & (table['Sell'][index - 1] == 'Closed')): # we're in a closed postion
                table['Profit'][index] = 0 + table['Profit'][index]     # add to profit just incase it's not zero
            elif ((table['Sell'][index] == 'Closed') & (table['Sell'][index - 1] == 'Open')): # we just closed a postition
                closed_price = table[' Close'][index]
                table['Profit'][index] = (open_price - closed_price) + table['Profit'][index]
            elif ((table['Sell'][index] == 'Open') & (table['Sell'][index - 1] == 'Open')): # we're in an open postion
                table['Profit'][index] = 0 + table['Profit'][index]
            elif ((table['Sell'][index] == 'Open') & (table['Sell'][index - 1] == 'Closed')): # we just opend a postion
                open_price = table[' Close'][index]

    return table['Profit']

In [20]:
eur_usd_df['Profit'] = profit(eur_usd_df)
cad_usd_df['Profit'] = profit(cad_usd_df)
jpy_usd_df['Profit'] = profit(jpy_usd_df)

/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_69551/1398871199.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Profit'][index] = 0
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_69551/1398871199.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table['Profit'][index] = 0
/var/folders/hc/nkkc_9yd3fd9x9025zg61t2w0000gn/T/ipykernel_69551/1398871199.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  t

In [21]:
conn = sqlite3.connect('test.sqlite')

In [22]:
eur_usd_df.to_sql('eur_usd', conn, if_exists='replace', index=False)
cad_usd_df.to_sql('cad_usd', conn, if_exists='replace', index=False)
jpy_usd_df.to_sql('jpy_usd', conn, if_exists='replace', index=False)

263

In [23]:
conn.close()